<a href="https://colab.research.google.com/github/MoMus2000/Airline-Sentiment-Analysis/blob/master/AirlineSentimentpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import collections
import re
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Bidirectional,LSTM,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
import sklearn
from bs4 import BeautifulSoup
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
df = pd.read_csv('/content/Tweets (1).csv')
# Good practice to shuffle data before use
df = df.reindex(np.random.permutation(df.index))
df = df[['text','airline_sentiment']]


In [4]:
from nltk.corpus import stopwords

X = df

X["text"] = X["text"].apply(lambda x: BeautifulSoup(x, "lxml").get_text())
X["text"] = X["text"].apply(lambda x: x.lower())
X["text"] = X["text"].apply(lambda x: re.sub("[^a-zA-Z]", " ", x))
X["text"] = X["text"].apply(lambda x: re.sub("\s+", " ", x))


y = X["airline_sentiment"]
X = X["text"]

stopwords = stopwords.words('english')
# vectorizer = TfidfVectorizer()
vectorizer = sklearn.feature_extraction.text.CountVectorizer(stop_words=stopwords)

X = vectorizer.fit_transform(X)

In [5]:
X = df.text
Y = df.airline_sentiment

label_encoder = LabelEncoder()

Y = label_encoder.fit_transform(Y)

Y = to_categorical(Y)

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15)

max_words = len(set(" ".join(X_train).split()))
max_len = X_train.apply(lambda x: len(x)).max()


In [9]:
tokenizer = Tokenizer(num_words=max_words)

tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_train_seq = tf.keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen=max_len)

In [43]:
model = Sequential()
model.add(tf.keras.layers.Embedding(max_words,100,input_length=max_len))
model.add(tf.keras.layers.Flatten())
model.add(Dropout(0.5))
model.add(Dense(20, activation = 'relu'))
model.add(Dense(20, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

In [44]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


In [45]:
model.fit(X_train_seq,Y_train,validation_split = 0.2,epochs=50)

Epoch 1/50
312/312 [==============================] - 5s 17ms/step - loss: 0.8037 - accuracy: 0.6545 - val_loss: 0.6500 - val_accuracy: 0.7336
Epoch 2/50
312/312 [==============================] - 5s 16ms/step - loss: 0.5800 - accuracy: 0.7489 - val_loss: 0.6028 - val_accuracy: 0.7609
Epoch 3/50
312/312 [==============================] - 5s 16ms/step - loss: 0.4166 - accuracy: 0.8387 - val_loss: 0.6102 - val_accuracy: 0.7843
Epoch 4/50
312/312 [==============================] - 5s 16ms/step - loss: 0.2759 - accuracy: 0.9026 - val_loss: 0.7047 - val_accuracy: 0.7871
Epoch 5/50
312/312 [==============================] - 5s 16ms/step - loss: 0.1844 - accuracy: 0.9385 - val_loss: 0.7702 - val_accuracy: 0.7758
Epoch 6/50
312/312 [==============================] - 5s 16ms/step - loss: 0.1297 - accuracy: 0.9585 - val_loss: 0.8488 - val_accuracy: 0.7826
Epoch 7/50
312/312 [==============================] - 5s 16ms/step - loss: 0.1010 - accuracy: 0.9699 - val_loss: 0.8856 - val_accuracy: 0.7738

KeyboardInterrupt: ignored